Ce TP est en deux parties


*   La première consiste à programmer quelques méthodes de classification multilabel
*   La seconde consiste à benchmarker des méthodes de classific aiton multilael sur un jeu de données, en utiisant un package dédié à la classification multilabels



# 1. Programmation de méthodes de classification multilabel

## Importing required packages

In [84]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.base import clone

## 0. Load the Yeast Dataset

In [123]:
yeast = pd.read_csv('yeast.csv')

In [124]:
X = yeast.iloc[:, 0:-14]

In [125]:
y = yeast.iloc[:,-14:]

In [126]:
# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = 0.7)

In [89]:
y_train

,Class1,Class2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,Class13,Class14
2067,1,0,0,0,0,0,0,0,0,0,0,0,0,0
936,0,0,1,1,0,0,0,0,0,0,0,1,1,0
376,0,1,1,0,0,0,1,1,1,0,0,0,0,0
2353,0,0,0,0,0,0,1,1,0,0,0,1,1,0
1432,0,0,1,1,0,0,1,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,1,1,1,1,0,0,0,0,0,0,0,1,1,0
1731,0,1,1,0,0,0,0,0,0,0,0,0,0,0
763,1,1,1,1,0,0,0,0,0,0,0,1,1,0
835,0,0,1,1,0,0,0,0,0,0,0,1,1,0


In [90]:
y_train_array = y_train.to_numpy()
y_train_array

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 1, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 0],
       [0, 0, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [91]:
print(y_train_array.shape)

(1691, 14)


## Définition de métriques

Que calculent les métriques suivantes ?
-> L'accuracy score compare les prédictions réalise par le model avec les vrais étiquettes et retourn un score
-> La distance de Hamming calcule le nombre de bit différents entre deux nombres binaires

In [92]:
# Custom accuracy score

def get_accuracy_score(y_pred,y_test):

    if 'numpy' not in str(type(y_pred)):
        y_pred = y_pred.to_numpy()

    if 'numpy' not in str(type(y_test)):
        y_test = y_test.to_numpy()


    assert(y_test.shape == y_pred.shape)

    if y_pred.shape[1] <= 5: #For a smaller number of labels, a ratio of half the labels being correct is good enough
        ratio = 0.5
    else:
        ratio = 0.7 #For a number of labels, at least 70% of the predicted labels must be correct

    acc_rows = []

    for i in range(len(y_test)):
        acc_rows.append(np.count_nonzero(y_test[i]==y_pred[i]))

    acc_rows = [1 if x/y_pred.shape[1] >= ratio else 0 for x in acc_rows] #1 if ratio of match in a row is greater than ratio, else 0
    return sum(acc_rows)/len(acc_rows)

In [93]:
### Distance de Hamming

def HammingDistance_score(y_pred,y_test):

    if 'numpy' not in str(type(y_pred)):
        y_pred = y_pred.to_numpy()

    if 'numpy' not in str(type(y_test)):
        y_test = y_test.to_numpy()

    acc_rows = []

    for i in range(len(y_test)):
      acc_rows.append(np.count_nonzero(y_test[i]==y_pred[i]))

    return sum(acc_rows)/len(acc_rows)


## 1. Implement the Binary Relevance Algorithm

Implémentez la classe BinaryRelevanceClassifier suivante qui prend en paramètre un estimateur et va l'utiliser sur chacun des labels qe ce soit en apprentissage ou en inférence.
* Vous stockerez les modèles dans une liste *model_list*
* Il est inutile d'implémenter la méthode *predict_proba*

In [94]:
class BinaryRelevanceClassifier(BaseEstimator, ClassifierMixin):

    def __init__(self, base_model=LogisticRegression()):
        self.base_model = base_model #base model - by default logistic regression

    def fit(self, X, y):
        self.model_list_ = []
        y = np.array(y)
        for i in range(y.shape[1]):
            model = clone(self.base_model)
            model.fit(X, y[:,i])
            self.model_list_.append(model)
        return self

    def predict(self,X):
        pred = []
        for model in self.model_list_:
            preds = model.predict(X)
            pred.append(preds)
        y_pred = np.array(pred).T
        return y_pred

    def predict_proba(self,X):
        return None

Le code de la cellule suivante doit fonctionner si la classe BinaryRelevance ci-dessus est ok.

In [95]:
binclf = BinaryRelevanceClassifier(base_model=SVC(C=1))
binclf.fit(X_train,y_train)
y_pred = binclf.predict(X_test)

In [96]:
display('Accuracy : ',get_accuracy_score(y_test,y_pred), 'Hamming Distance : ', HammingDistance_score(y_test,y_pred) )
display('F1 score : ',f1_score(y_test, y_pred, average='macro'))

'Accuracy : '

0.8181818181818182

'Hamming Distance : '

11.297520661157025

'F1 score : '

np.float64(0.3629736002564711)

# 2. Implement the label set method

## Construction des labels

In [97]:
# le code ci-dessous recupère les données y dans un tableau numpy.

import numpy as np

# Convert the pandas DataFrame 'y' to a NumPy array.
y_array = y.to_numpy()

# Print the resulting NumPy array.
y_array

array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 1, 1, ..., 1, 1, 0]])

In [98]:
y_unique, index_unique, reverse_unique = np.unique(y_array,return_index=True, return_inverse=True, axis=0)

In [99]:
print (y_unique.shape, y_unique[0])

(198, 14) [0 0 0 0 0 0 0 0 0 0 0 1 1 0]


In [100]:
index_unique

array([ 108, 1087,  878,  111, 1793,    5, 1053,   14,   84,  570, 1686,
        152,    0, 2064,  617,  261,  508,  170,   38,  664,  739, 1556,
        467,  468, 2362,   66,   54, 1192,   46,   16, 1558,  115, 1163,
         45,  441,  868, 1315,   63,   10,   27,   23,  538,   47,   86,
         13,  329,  117, 1889,  141, 2128,  148,    1,  611,    9,  397,
       1208, 1945, 2212,   70,  712,   50, 2375,  653,  904,   71, 1704,
        424,  546,  466,  216,    4,   44,  930, 1227, 1792,  319, 1683,
         12,    2, 1665,   76,  138,  249,   25,  250,  287,  132,  305,
         61,  786, 1037,   36,  376, 1040, 1679,  970,  494,   24,  587,
         34,  550,   97, 1831,  882,  872,  894,  326,  891, 2182,  348,
         74,   48,  862, 2144,  671, 1443,  588,  568,  544,  289,  807,
       1819,  171,    6,  211,  731, 2233,   29, 1297,  362,  605,   56,
       1784,  229,  385,    7,   15,   33,    8, 1157,  236,  409,  338,
       2025,   65,  200,  184,  106, 2222,  251, 13

In [101]:
print (y_array[108,:])

[0 0 0 0 0 0 0 0 0 0 0 1 1 0]


In [102]:
print(reverse_unique[108])

0


Que contiennent les tableaux créés ci-dessus ?
-> y_unique contient les lignes unique de y_array
-> index_unique contient les indices des premières occurrences de chaque ligne unique dans y_array
-> reverse_unique contient pour chaque ligne de y_array l'indice correspondant dans y_unique

Construisez les données X_trainLS, X_testLS, y_trainLS, y_testLS pour la méthode Label Set.

In [103]:
X_trainLS, X_testLS, y_trainLS, y_testLS = train_test_split(X, reverse_unique, random_state=0, train_size=0.7)

## Expériences

La fonction `reconstruit_y` ci-dessous reconstruit les prédictions du modèle appris avec la méthode Label Set au format attendu par les métriques de performance définies plus haut. Expliquez le fonctionnement de cette fonction.

In [81]:
def reconstruit_y(yLS,  LS ): # reconstruit les vecteurs y (vecteurs d'indicateurs de labels yLS) à partir des LS (les labelsets)
  y_temp = [LS[ yLS[i]] for i in range(len(yLS))]
  y_temp = np.concatenate( y_temp, axis=0 )
  y_temp = y_temp.reshape((yLS.shape[0],LS.shape[1]))
  return y_temp

Faites quelques expériences avec la méthode Label Set.

In [106]:
clf_LS = LogisticRegression(max_iter=1000)
clf_LS.fit(X_trainLS, y_trainLS)

y_predLS = clf_LS.predict(X_testLS)

y_pred_reconstructed = reconstruit_y(y_predLS, y_unique)
y_test_reconstructed = reconstruit_y(y_testLS, y_unique)

In [107]:
print("Accuracy : ", get_accuracy_score(y_pred_reconstructed, y_test_reconstructed))
print("Hamming Distance : ", HammingDistance_score(y_pred_reconstructed, y_test_reconstructed))
print("F1 Score : ", f1_score(y_test_reconstructed, y_pred_reconstructed, average='macro'))

Accuracy :  0.7658402203856749
Hamming Distance :  11.017906336088155
F1 Score :  0.3769438722563896


## Task 3: Implement the Classifier Chains Algorithm

Complétez la classe suivante qui implémente les classifier Chains.


In [108]:
class ClassifierChains(BaseEstimator, ClassifierMixin):

    def __init__(self, base_model=LogisticRegression(), order=None):
        self.base_model = base_model
        self.order = order

    def fit(self, X, y):
        X_cpy = X.to_numpy()
        y_cpy = y.to_numpy()

        if self.order is None:
            self.order_ = np.arange(y_cpy.shape[1])
        else:
            self.order_ = np.array(self.order)

        self.model_list_ = []

        X_train_chain = X_cpy
        for label_index in self.order_:
            model = clone(self.base_model)
            model.fit(X_train_chain, y_cpy[:, label_index])
            self.model_list_.append(model)
            X_train_chain = np.column_stack((X_train_chain, y_cpy[:, label_index]))

        return self

    def predict(self, X):
        X_test_chain = X.to_numpy()
        y_pred_chain = []

        for i, model in enumerate(self.model_list_):
            y_pred_i = model.predict(X_test_chain)
            y_pred_chain.append(y_pred_i)
            X_test_chain = np.column_stack((X_test_chain, y_pred_i))


        y_pred_matrix = np.array(y_pred_chain).T

        inv_order = np.argsort(self.order_)
        y_pred_matrix = y_pred_matrix[:, inv_order]

        return y_pred_matrix

    def predict_proba(self, X):
        return None

Faites des expériences avec la méthode Classifer Chain

In [109]:
chain_clf = ClassifierChains(base_model=LogisticRegression(max_iter=1000))
chain_clf.fit(X_train, y_train)
y_pred_chain = chain_clf.predict(X_test)

print("Accuracy : ", get_accuracy_score(y_pred_chain, y_test))
print("Hamming Distance : ", HammingDistance_score(y_pred_chain, y_test))
print("F1 Score : ", f1_score(y_test, y_pred_chain, average='macro'))

Accuracy :  0.78099173553719
Hamming Distance :  10.980716253443527
F1 Score :  0.38017228495944805


# 2. Utilisation du package [scikit-multilearn](http://scikit.ml/modelselection.html)

Installez ce package et utilisez le pour reproduire des expériences ci-dessus sur les mêmes données.

Vosu ferez des expériences avec BR, Classifier Chain mais et des estimateurs de votre choix mais aussi avec de sméthodes naturellement transformées comme les KNN (MLKNN en version multilabel)

A noter que le code du package se trouve [ici](https://github.com/scikit-multilearn/scikit-multilearn/tree/master/skmultilearn) et qu'un bug est idenfitié sur la méthode MLKNN à corriger selon [ces indications](https://stackoverflow.com/questions/74613688/typeerror-skmultilearn-error-with-multilabel-knn) en redéfinissant la classe MLKNN.

Vous pourrez chercher les meileurs hyperparamètres avec un code tel que celui-ci pour le cas des MLKNNs.


```
from sklearn.model_selection import GridSearchCV


parameters = {'k': range(1,20)}
score = 'f1_weighted'

classifier = GridSearchCV(MLkNN(), parameters, scoring=score)
classifier.fit(X, y)

classifier.best_params_, classifier.best_score_
```



In [135]:
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain
from skmultilearn.adapt import MLkNN

In [140]:
mlknn_clf = MLkNN()
mlknn_clf.fit(X_train, y_train)
y_pred_mlknn = mlknn_clf.predict(X_test)


TypeError: NearestNeighbors.__init__() takes 1 positional argument but 2 were given